<a href="https://colab.research.google.com/github/IzaakGagnon/Integrated_Information_Testing/blob/main/Custom_Partitioning_Schemes_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports and Installs

In [8]:
!python -m pip install -U git+https://github.com/wmayner/pyphi.git@feature/iit-4.0
import numpy as np
import pyphi
import random
import time
pyphi.config.PROGRESS_BARS = False
pyphi.config.PARALLEL = False
pyphi.config.SHORTCIRCUIT_SIA = False
pyphi.config.WELCOME_OFF = True
pyphi.config.REPERTOIRE_DISTANCE = "GENERALIZED_INTRINSIC_DIFFERENCE"

  Cloning https://github.com/wmayner/pyphi.git (to revision feature/iit-4.0) to /tmp/pip-req-build-nd023za2
  Running command git clone --filter=blob:none --quiet https://github.com/wmayner/pyphi.git /tmp/pip-req-build-nd023za2
  Resolved https://github.com/wmayner/pyphi.git to commit 6b83cbdbbcdca75289415fe096adbac5f2ec7a4d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.6/662.6 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 21.8 MB/s eta 0:00:00
     ━━━━━━

In [ ]:
import functools
import itertools
from itertools import chain, product
import numpy as np
from more_itertools import distinct_permutations
from toolz import unique
from pyphi import combinatorics
from pyphi.cache import cache
from pyphi.conf import config, fallback
from pyphi.direction import Direction
from pyphi.models.cuts import (
    Bipartition,
    CompleteGeneralKCut,
    CompleteGeneralSetPartition,
    Cut,
    GeneralKCut,
    GeneralSetPartition,
    KPartition,
    Part,
    SystemPartition,
    Tripartition,
)
from pyphi.partition import system_partition_types
from pyphi.registry import Registry
from itertools import combinations

In [ ]:
#### NETWORK GENERATING FUNCTIONS
def normalize_rows(matrix):
    ### Scales probabilities in a matrix so that they satisfy the markov property.
    num_rows = matrix.shape[0]
    normalized_matrix = np.zeros_like(matrix)  # Create an empty matrix of the same shape
    for i in range(num_rows):
        row = matrix[i, :]
        current_sum = np.sum(row)
        if current_sum > 0:
            scaling_factor = 1.0 / current_sum
            normalized_row = row * scaling_factor
            normalized_matrix[i, :] = normalized_row
        else: print("Zero_Sum_Error: Problem With Values Generated")
    return normalized_matrix
def create_noisy_network(size):
    ### Obtains a random network of nodes of a given size, directly ready for computing Phi.
    return pyphi.Network(normalize_rows(pyphi.convert.state_by_node2state_by_state(np.random.rand(2**size,size))))
def create_noiseless_network(size):
  return pyphi.Network(np.random.randint(0,2,(2**size,size)))

In [112]:
#####   Implementation of all Custom Partitioning Schemes


#####                                                               #####
#####                                                               #####
##### Functions to Support the Faster Implementation of SET_UNI/BI  #####
#####                                                               #####
#####                                                               #####


def gen_seqs(partition):

     ### Generates all unique sequences of cuts for a given partition
     ### These sequences represent possible cut-directions between partitions, used in constructing unique cut matricies
     ### Proven that a sequence of cuts with one part having inputs cut, and another part having outputs cut will lead to a unique cm

    n = len(partition)
    elements = [0,1,2] #  0 = cut inputs, 1 = cut outputs, 2 = cut both
    all_sequences = itertools.product(elements, repeat=n)
    for seq in all_sequences:
        if (0 in seq) and (1 in seq):
            yield seq
    yield (2,)*n ### Yield the full cut




def seq_2_cm(seq, partition,_node_indices):


   ### Constructs the cut matrix from such a unique sequence -See gen_seqs()
   ### Considers elements representing cut directions in a sequence, and updates such cuts to the cut matrix.


  cut_matrix = np.zeros([len(_node_indices), len(_node_indices)], dtype=int)


  for part,s in zip(partition,seq):
    nonpart = list(_node_indices - set(part))
    if s == 0:                 ### If we are cutting the part's inputs
      source, target = nonpart, part
    else:                      ### If we are cutting the part's outputs
      source, target = part, nonpart
    cut_matrix[np.ix_(source, target)] = 1
    if s == 2:                   ### If we are cutting both inputs and outputs
      cut_matrix[np.ix_(target, source)] = 1
  return cut_matrix




### Full implementation of the faster SET_UNI/BI Algorithm
@system_partition_types.register("ALL_K-PARTITIONS")
def generate_all_partitions(node_indices, node_labels=None):


  ### Perform the trivial cut
  if len(node_indices) == 1 or config.SYSTEM_PARTITION_INCLUDE_COMPLETE:
        yield CompleteGeneralSetPartition(node_indices, node_labels=node_labels)

  _node_indices = set(range(len(node_indices)))


  ### Loop through the possible partitions of the network.
  for partition in combinatorics.set_partitions(_node_indices, nontrivial=True):

    ### Generate all of the possible direction-cut combinations between parts
    for seq in gen_seqs(partition):
      cm = seq_2_cm(seq, partition,_node_indices)
      yield GeneralSetPartition(
                node_indices,
                cm,
                node_labels=node_labels,
                set_partition=partition,)





#####                                                               #####
#####                                                               #####
##### Functions To Support The Faster Generation Of Bipartitions    #####
#####                                                               #####
#####                                                               #####



### Generate All Possible Bi-Partitions Of A Set Of Nodes
def get_bipartitions(collection):
    collection = list(collection)
    n = len(collection)
    # Special case: no bipartitions possible if the collection has fewer than 2 elements
    if n < 2:
        return
    # Generate all combinations for the first subset of size from 1 to n-1
    for i in range(1, n):
        for first_subset in combinations(collection, i):
            second_subset = [item for item in collection if item not in first_subset]
            yield [list(first_subset), second_subset]




### Full Implementation Of The Faster Bi-Partition Algorithm
@system_partition_types.register("ALL_BI-PARTITIONS")
def generate_all_bipartitions(node_indices, node_labels=None):


  ### Perform the trivial cut
  if len(node_indices) == 1 or config.SYSTEM_PARTITION_INCLUDE_COMPLETE:
        yield CompleteGeneralSetPartition(node_indices, node_labels=node_labels)

  _node_indices = set(range(len(node_indices)))


  ### Loop through the possible partitions of the network.
  for partition in get_bipartitions(_node_indices):

    ### Generate all of the possible direction-cut combinations between parts
    for seq in gen_seqs(partition):
      cm = seq_2_cm(seq, partition,_node_indices) ## See Functions used in K-Partitions Generator For Information
      yield GeneralSetPartition(
                node_indices,
                cm,
                node_labels=node_labels,
                set_partition=partition,)


#####                                                               #####
#####                                                               #####
##### Functions To Support The Faster Generation Of Random K_Parts  #####
#####                                                               #####
#####                                                               #####

def stirling2(n,k):

    ### Computes Stirling's Number of the Second Kind S2(n,k)

    n1=n
    k1=k
    if n<=0:
        return 1
    elif k<=0:
        return 0
    elif (n==0 and k==0):
        return -1
    elif n!=0 and n==k:
        return 1
    elif n<k:
        return 0
    else:
        temp1=stirling2(n1-1,k1)
        temp1=k1*temp1
        return (k1*(stirling2(n1-1,k1)))+stirling2(n1-1,k1-1)



def total_partition_count(size):

  ### A Counter for the total number of k-partitions


  a = 0
  for i in range(1,size + 1):
    a += stirling2(size,i) * ( 3**i - 2**(i + 1) + 2)
  return a



### Full implementation of the random k-partition method
### Remember to define n as needed
### TODO: Refine with faster algorithm
@system_partition_types.register("RANDOM_K_PARTITIONS")
def generate_random_partitions(node_indices, node_labels=None):
  if len(node_indices) == 1 or config.SYSTEM_PARTITION_INCLUDE_COMPLETE:
        yield CompleteGeneralSetPartition(node_indices, node_labels=node_labels)
  _node_indices = set(range(len(node_indices)))
  total = total_partition_count(len(_node_indices))
  count = 0
  sample = random.sample(range(0,total + 1),n)
  for partition in combinatorics.set_partitions(_node_indices, nontrivial=True):
    for s in gen_seqs(partition):
      if count in sample:
        cm = seq_2_cm(s, partition,_node_indices)
        yield GeneralSetPartition(
                  node_indices,
                  cm,
                  node_labels=node_labels,
                  set_partition=partition,)
      count +=1



#####                                                               #####
#####                                                               #####
##### Functions To Support The Faster Generation Of Random Bi-Parts #####
#####                                                               #####
#####                                                               #####

def total_bipartition_count(size):
  return stirling2(size,2)*3
@system_partition_types.register("RANDOM_BI_PARTITIONS")
def generate_random_bipartitions(node_indices, node_labels=None):
  if len(node_indices) == 1 or config.SYSTEM_PARTITION_INCLUDE_COMPLETE:
        yield CompleteGeneralSetPartition(node_indices, node_labels=node_labels)
  _node_indices = set(range(len(node_indices)))
  total = total_bipartition_count(len(_node_indices))
  count = 0
  sample = random.sample(range(0,total + 1),n)
  for partition in bipartitions(_node_indices):
    for s in gen_seqs(partition):
      if count in sample:
        cm = seq_2_cm(s, partition,_node_indices)
        yield GeneralSetPartition(
                  node_indices,
                  cm,
                  node_labels=node_labels,
                  set_partition=partition,)
      count +=1




#####                                                               #####
#####                                                               #####
##### Functions To Support The Uni-Directional Approximation Method #####
#####                                                               #####
#####                                                               #####


def gen_seqs_no_bi(partition):


    ### Generates all sequences of (inputs, outputs) ~ direction of cuts for parts
    ### Similar to gen_seqs() but with 2 possible cut directions instead of 3


    n = len(partition)
    elements = [True, False]
    all_sequences = itertools.product(elements, repeat=n)
    for seq in all_sequences:
        yield seq


def seq_2_cm_no_bi(seq, partition, _node_indices):


    ### Constructs the cut matrix from such a unique sequence -See gen_seqs_no_bi()
    ### Considers elements representing cut directions in a sequence, and updates such cuts to the cut matrix.

    cut_matrix = np.zeros([len(_node_indices), len(_node_indices)], dtype=int)
    for part, s in zip(partition, seq):
        nonpart = list(_node_indices - set(part))
        if s:  # If we are cutting the part's inputs
            cut_matrix[np.ix_(nonpart, part)] = 1
        else:  # If we are cutting the part's outputs
            cut_matrix[np.ix_(part, nonpart)] = 1
    return cut_matrix

### Full Implementation of the Unidirectional Approximation Method
@system_partition_types.register("NO_BI_DIRECTIONAL_CUTS_APPROXIMATION")
def generate_all_partitions(node_indices, node_labels=None):
    if len(node_indices) == 1 or config.SYSTEM_PARTITION_INCLUDE_COMPLETE:
        yield CompleteGeneralSetPartition(node_indices, node_labels=node_labels)

    _node_indices = set(range(len(node_indices)))
    for partition in combinatorics.set_partitions(_node_indices, nontrivial=True):
        new_var = gen_seqs_no_bi(partition)
        for s in new_var:
            cm = seq_2_cm_no_bi(s, partition, _node_indices)
            yield GeneralSetPartition(
                node_indices,
                cm,
                node_labels=node_labels,
                set_partition=partition,
            )


In [116]:
size = 4
network = create_noisy_network(size)
state = [np.random.choice([0, 1]) for i in range(size)]
subsystem_cause = pyphi.Subsystem(network,state,backward_tpm=True)
subsystem_effect = pyphi.Subsystem(network,state,backward_tpm=False)

pyphi.config.SYSTEM_PARTITION_TYPE = "SET_UNI/BI" #### The Original
s1 = time.time()
sia1 = pyphi.backwards.sia(subsystem_cause,subsystem_effect)
f1 = time.time()
print(sia1)
print(f1 - s1,pyphi.config.SYSTEM_PARTITION_TYPE)

pyphi.config.SYSTEM_PARTITION_TYPE = "ALL_K-PARTITIONS" #### Faster than original
s2 = time.time()
sia2 = pyphi.backwards.sia(subsystem_cause,subsystem_effect)
f2 = time.time()
print(sia2)
print(f2 - s2,pyphi.config.SYSTEM_PARTITION_TYPE)

pyphi.config.SYSTEM_PARTITION_TYPE = "ALL_BI-PARTITIONS" #### Approximation one
s3 = time.time()
sia3 = pyphi.backwards.sia(subsystem_cause,subsystem_effect)
f3 = time.time()
print(sia3)
print(f3 - s3,pyphi.config.SYSTEM_PARTITION_TYPE)

pyphi.config.SYSTEM_PARTITION_TYPE = "RANDOM_K_PARTITIONS" #### Approximation two
n = 10
s4 = time.time()
sia4 = pyphi.backwards.sia(subsystem_cause,subsystem_effect)
f4 = time.time()
print(sia4)
print(f4 - s4,pyphi.config.SYSTEM_PARTITION_TYPE)

pyphi.config.SYSTEM_PARTITION_TYPE = "RANDOM_BI_PARTITIONS" ### Approximation one and two
s5 = time.time()
sia5 = pyphi.backwards.sia(subsystem_cause,subsystem_effect)
f5 = time.time()
print(sia5)
print(f5 - s5,pyphi.config.SYSTEM_PARTITION_TYPE)

pyphi.config.SYSTEM_PARTITION_TYPE = "NO_BI_DIRECTIONAL_CUTS_APPROXIMATION" ### Approximation three
s6 = time.time()
sia6 = pyphi.backwards.sia(subsystem_cause,subsystem_effect)
f6 = time.time()
print(sia6)
print(f6 - s6,pyphi.config.SYSTEM_PARTITION_TYPE)

┌────────────────────────────────────────┐
│      SystemIrreducibilityAnalysis      │
│ ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ │
│      Subsystem:  n0,n1,n2,n3           │
│  Current state:  (1,0,1,1)             │
│            φ_s: 9.698408326634024e-05  │
│ Normalized φ_s: 3.2328027755446744e-05 │
│          CAUSE:  (0,0,1,1)             │
│           II_c: 0.8929088586256291     │
│         EFFECT:  (0,0,0,0)             │
│           II_e: 1.136999116026381      │
│   #(tied MIPs): 0                      │
│      Partition:                        │
│                 2 parts: {n1,n0n2n3}   │
│                 [[0 1 0 0]             │
│                  [0 0 0 0]             │
│                  [0 1 0 0]             │
│                  [0 1 0 0]]            │
└────────────────────────────────────────┘
1.8952388763427734 SET_UNI/BI
┌────────────────────────────────────────┐
│      SystemIrreducibilityAnalysis      │
│ ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ │
│      Subsystem:  n0,n1